In [2]:
import pyodbc
import pandas as pd
server = 'romcapurro.database.windows.net'
database = 'telecom'
username = 'romcapurro'
password = '{C!rc4C0mb4t}'
driver= '{ODBC Driver 18 for SQL Server}'

# Crear un DataFrame de ejemplo
df = pd.read_csv('historico_velocidad_internet_media_nuevo.csv')
df = df.drop(columns=['Unnamed: 0'])

# Cadena de conexión
conn_str = f'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'

# Conexion
conn = pyodbc.connect(conn_str)

# Crear un cursor
cursor = conn.cursor()

# Nombre de la tabla en Azure SQL Database
table_name = 'historico_velocidad_media'

# Crear la tabla si no existe (puedes personalizar esto según tus necesidades)
cursor.execute(f'''
    IF OBJECT_ID('{table_name}', 'U') IS NULL
    BEGIN
        CREATE TABLE {table_name} (
            anio INT,
            trimestre INT,
            provincia VARCHAR(100),
            velocidad_media VARCHAR(10)
        );
    END
''')
conn.commit()

# Insertar datos en la tabla
for index, row in df.iterrows():
    cursor.execute(f"INSERT INTO {table_name} (anio, trimestre, provincia, velocidad_media) VALUES ({row['Año']}, {row['Trimestre']}, '{row['Provincia']}', '{row['Velocidad_Media']}')")

# Confirmar la transacción
conn.commit()

# Cerrar la conexión
conn.close()